In [ ]:
%run sources/sequence_dataset.py
%run sources/sequence_walking_tree.py

In [ ]:
def read_event_logs(csv_path):
    r = pd.read_csv(csv_path)
    return list(r["sequence"].apply(lambda x: eval(x) )), list(r["class"])

X,Y = read_event_logs("datasets/20240124_0525_fitbit_p01_quartiles.csv")

In [ ]:
Z = SequenceDataset()
Z.fit(X,Y)

In [ ]:
from math import log2
mh = int(log2(Z.last_position_df.last_position.mean()))
ms = int(log2(len(Z.raw_x)))
mh,ms

In [ ]:
C = SequenceWalkingTree(max_height=100, min_node_samples=0, alternate_if_possible=False, random_samples=100)
t = C.fit(Z)

In [ ]:
from networkx.drawing.nx_pydot import to_pydot
from IPython.display import Image, display
import networkx as nx


def generate_tree(g, t, k=0):
    newline = "\n"
    g.add_node(k, label=f'{t["test_type"] if "test_type" in t else ""}{t["test"] if "test" in t else ""}{newline + "true_samples - "+str(t["true_samples"])+ " " if "true_samples" in t else "" }{newline+"false_samples - "+str(t["false_samples"]) if "false_samples" in t else "" }{newline+str(t["probabilities"])}'  )
    next_k = k + 1
    for key in [True, False]:
        if key in t:
            current_k, next_k = generate_tree(g, t[key], next_k)
            g.add_edge(k, current_k, label= f'{key}')
    return k, next_k

G = nx.DiGraph()

generate_tree(G,C.tree)

pydot_graph = to_pydot(G)
dot_file_path = "tree.dot"
pydot_graph.write_dot(dot_file_path)

!dot -Tpng tree.dot -o tree.png

display(Image("tree.png"))